# Imported CWL types

Follow below an example to demonstrate how the library handles and convert a CWL where inputs are imported from an external URL.

## 1. Parsing

Convert the CWL document, from its YAML representation, to the CWL DOM via [cwl-utils APIs](https://github.com/common-workflow-language/cwl-utils):

In [1]:
from cwl_utils.parser import load_document_by_yaml, save
import yaml

with open('./workflows/imported-cwl-types.cwl') as f:
    cwl_content = yaml.load(f, Loader=yaml.SafeLoader)

cwl = load_document_by_yaml(yaml=cwl_content, uri="io://", load_all=True)

print(
    yaml.dump(
        save(
            val=cwl,
            relative_uris=False
        )
    )
)

baseCommand: echo
class: CommandLineTool
cwlVersion: v1.2
id: io:/#main
inputs:
- doc: Area of interest defined as a bounding box
  id: io:/#main/aoi
  inputBinding:
    valueFrom: "${\n  /* Validate the length of bbox to be either 4 or 6 */\n  var\
      \ aoi = JSON.parse(self.contents);\n  var bboxLength = aoi.bbox.length;\n  if\
      \ (bboxLength !== 4 && bboxLength !== 6) {\n    throw \"Invalid bbox length:\
      \ bbox must have either 4 or 6 elements.\";\n  }\n  /* Convert bbox array to\
      \ a space-separated string for echo */\n  return aoi.bbox.join(' ') + \" CRS:\
      \ \" + aoi.crs;\n}\n"
  label: Area of interest
  loadContents: true
  type: https://raw.githubusercontent.com/eoap/schemas/main/geojson.yaml#FeatureCollection
label: Echo OGC BBox
outputs:
- id: io:/#main/echo_output
  type: stdout
requirements:
- class: InlineJavascriptRequirement
- class: SchemaDefRequirement
  types:
  - name: https://raw.githubusercontent.com/eoap/schemas/main/geojson.yaml#PointTyp

# 2. Inputs conversion

Once the document is parsed, invoke the `cwl2ogc` APIs to convert the CWL inputs to the OGC JSON format:

In [2]:
from cwl2ogc import BaseCWLtypes2OGCConverter
import json

converter = BaseCWLtypes2OGCConverter(cwl)

inputs = converter.to_ogc(cwl.inputs)

print(json.dumps(inputs, indent=2))

{
  "aoi": {
    "schema": {
      "type": "object",
      "properties": {
        "type": {
          "type": "string",
          "enum": [
            "FeatureCollection"
          ]
        },
        "features": {
          "type": "array",
          "items": {
            "type": "object",
            "properties": {
              "type": {
                "type": "string",
                "enum": [
                  "Feature"
                ]
              },
              "id": {
                "type": "string"
              },
              "geometry": {
                "nullable": false,
                "anyOf": [
                  {
                    "type": "object",
                    "properties": {
                      "type": {
                        "type": "string",
                        "enum": [
                          "Point"
                        ]
                      },
                      "coordinates": {
                        "type": "array",


# 2. Outputs conversion

Users can reuse the `BaseCWLtypes2OGCConverter` instance to convert the CWL outputs to the OGC JSON format:

In [3]:
outputs = converter.to_ogc(cwl.outputs)

print(json.dumps(outputs, indent=2))

{
  "echo_output": {
    "schema": {
      "type": "string"
    }
  }
}
